In [1]:
import os
import glob

# Get the absolute path of the current project directory
project_dir = os.path.abspath('.')

# Get the parent of the parent directory
WORK_DIR = os.path.abspath(os.path.join(project_dir, '../../'))

# Change the working directory to the parent of the parent directory
os.chdir(WORK_DIR)

# Verify the change by printing the current working directory
print("Current Working Directory:", os.getcwd())

Current Working Directory: /Users/david.amat/Documents/david/pdf-search-llm-rag


# process_pdf_file

In [21]:
import PyPDF2
import pandas as pd
from typing import List

In [7]:
file_title = "attention_is_all_you_need"
pdf_path = f"data/{file_title}.pdf"
pdf_reader = PyPDF2.PdfReader(pdf_path)

In [8]:
pages_amount = len(pdf_reader.pages)
print(f'Amount of pages: {pages_amount}')

Amount of pages: 15


In [9]:
document_text = []

In [10]:
for i, page in enumerate(pdf_reader.pages):
    document_text.append([file_title, i+1, page.extract_text()])

In [14]:
# print(document_text[1][2])

In [93]:
columns = ["file_name", "page_number", "text"]
df_text = pd.DataFrame(document_text, columns=columns)

In [94]:
df_text

,file_name,page_number,text
0,attention_is_all_you_need,1,"Provided proper attribution is provided, Googl..."
1,attention_is_all_you_need,2,"1 Introduction\nRecurrent neural networks, lon..."
2,attention_is_all_you_need,3,Figure 1: The Transformer - model architecture...
3,attention_is_all_you_need,4,Scaled Dot-Product Attention\n Multi-Head Atte...
4,attention_is_all_you_need,5,output values. These are concatenated and once...
5,attention_is_all_you_need,6,"Table 1: Maximum path lengths, per-layer compl..."
6,attention_is_all_you_need,7,length nis smaller than the representation dim...
7,attention_is_all_you_need,8,Table 2: The Transformer achieves better BLEU ...
8,attention_is_all_you_need,9,Table 3: Variations on the Transformer archite...
9,attention_is_all_you_need,10,Table 4: The Transformer generalizes well to E...


In [41]:
page_num = 1
start_text = document_text[page_num][2][:25]
end_text = document_text[page_num][2][-25:]

In [44]:
document_text[page_num][2]

'1 Introduction\nRecurrent neural networks, long short-term memory [ 13] and gated recurrent [ 7] neural networks\nin particular, have been firmly established as state of the art approaches in sequence modeling and\ntransduction problems such as language modeling and machine translation [ 35,2,5]. Numerous\nefforts have since continued to push the boundaries of recurrent language models and encoder-decoder\narchitectures [38, 24, 15].\nRecurrent models typically factor computation along the symbol positions of the input and output\nsequences. Aligning the positions to steps in computation time, they generate a sequence of hidden\nstates ht, as a function of the previous hidden state ht−1and the input for position t. This inherently\nsequential nature precludes parallelization within training examples, which becomes critical at longer\nsequence lengths, as memory constraints limit batching across examples. Recent work has achieved\nsignificant improvements in computational efficiency th

In [43]:
start_text

'1 Introduction\nRecurrent '

In [42]:
print(start_text)
print(end_text)

1 Introduction
Recurrent 
en generating the next.
2


# process_text_data

In [45]:
def split_page(document: str) -> List[str]:
    return document.split('\n')

In [46]:
def get_paragraphs(data: pd.DataFrame) -> pd.DataFrame:
    # Explode the list to separate rows
    data_text_exploded = data.explode('text')

    # Add a 'paragraph' column indicating the index of the element in the list
    data_text_exploded['paragraph'] = data_text_exploded.groupby(
        ['file_name', 'page_number']
    ).cumcount() + 1

    return data_text_exploded

In [89]:
df = df_text.copy()

In [70]:
# Apply split_page function to split text into paragraphs
df['text'] = df['text'].apply(split_page)

In [71]:
for page_num in range(len(df['text'])):
    num_paragraphs = len(df['text'][page_num])
    print(f"Page {page_num}: {num_paragraphs} paragraphs")

Page 0: 50 paragraphs
Page 1: 50 paragraphs
Page 2: 23 paragraphs
Page 3: 35 paragraphs
Page 4: 49 paragraphs
Page 5: 47 paragraphs
Page 6: 45 paragraphs
Page 7: 46 paragraphs
Page 8: 51 paragraphs
Page 9: 48 paragraphs
Page 10: 45 paragraphs
Page 11: 43 paragraphs
Page 12: 73 paragraphs
Page 13: 113 paragraphs
Page 14: 113 paragraphs


In [74]:
data_text_exploded = df.explode('text')

In [76]:
data_text_exploded

,file_name,page_number,text
0,attention_is_all_you_need,1,"Provided proper attribution is provided, Googl..."
0,attention_is_all_you_need,1,reproduce the tables and figures in this paper...
0,attention_is_all_you_need,1,scholarly works.
0,attention_is_all_you_need,1,Attention Is All You Need
0,attention_is_all_you_need,1,Ashish Vaswani∗
...,...,...,...
14,attention_is_all_you_need,15,<EOS>
14,attention_is_all_you_need,15,<pad>Figure 5: Many of the attention heads exh...
14,attention_is_all_you_need,15,"sentence. We give two such examples above, fro..."
14,attention_is_all_you_need,15,at layer 5 of 6. The heads clearly learned to ...


In [77]:
data_text_exploded['paragraph'] = data_text_exploded.groupby(
    ['file_name', 'page_number']
).cumcount() + 1

In [78]:
data_text_exploded

,file_name,page_number,text,paragraph
0,attention_is_all_you_need,1,"Provided proper attribution is provided, Googl...",1
0,attention_is_all_you_need,1,reproduce the tables and figures in this paper...,2
0,attention_is_all_you_need,1,scholarly works.,3
0,attention_is_all_you_need,1,Attention Is All You Need,4
0,attention_is_all_you_need,1,Ashish Vaswani∗,5
...,...,...,...,...
14,attention_is_all_you_need,15,<EOS>,109
14,attention_is_all_you_need,15,<pad>Figure 5: Many of the attention heads exh...,110
14,attention_is_all_you_need,15,"sentence. We give two such examples above, fro...",111
14,attention_is_all_you_need,15,at layer 5 of 6. The heads clearly learned to ...,112


In [79]:
# Apply get_paragraphs function to explode the list and add paragraph numbers
df = get_paragraphs(df)

In [80]:
df

,file_name,page_number,text,paragraph
0,attention_is_all_you_need,1,"Provided proper attribution is provided, Googl...",1
0,attention_is_all_you_need,1,reproduce the tables and figures in this paper...,2
0,attention_is_all_you_need,1,scholarly works.,3
0,attention_is_all_you_need,1,Attention Is All You Need,4
0,attention_is_all_you_need,1,Ashish Vaswani∗,5
...,...,...,...,...
14,attention_is_all_you_need,15,<EOS>,109
14,attention_is_all_you_need,15,<pad>Figure 5: Many of the attention heads exh...,110
14,attention_is_all_you_need,15,"sentence. We give two such examples above, fro...",111
14,attention_is_all_you_need,15,at layer 5 of 6. The heads clearly learned to ...,112


In [81]:
# Apply strip to remove leading and trailing spaces
df['text'] = df['text'].str.strip()

In [84]:
# Reset index
df.reset_index(drop=True, inplace=True)

In [86]:
df = df[['file_name', 'page_number', 'paragraph', 'text']]

## Function

In [95]:
def process_text_data(df: pd.DataFrame) -> pd.DataFrame:
    # Apply split_page function to split text into paragraphs
    df['text'] = df['text'].apply(split_page)

    # Apply get_paragraphs function to explode the list and add paragraph numbers
    df = get_paragraphs(df)

    # Apply strip to remove leading and trailing spaces
    df['text'] = df['text'].str.strip()

    # Reset index
    df.reset_index(drop=True, inplace=True)

    # Reorder columns for better readability
    df = df[['file_name', 'page_number', 'paragraph', 'text']]

    return df

In [96]:
df = df_text.copy()
df_filtered = process_text_data(df)

In [97]:
df_filtered

,file_name,page_number,paragraph,text
0,attention_is_all_you_need,1,1,"Provided proper attribution is provided, Googl..."
1,attention_is_all_you_need,1,2,reproduce the tables and figures in this paper...
2,attention_is_all_you_need,1,3,scholarly works.
3,attention_is_all_you_need,1,4,Attention Is All You Need
4,attention_is_all_you_need,1,5,Ashish Vaswani∗
...,...,...,...,...
826,attention_is_all_you_need,15,109,<EOS>
827,attention_is_all_you_need,15,110,<pad>Figure 5: Many of the attention heads exh...
828,attention_is_all_you_need,15,111,"sentence. We give two such examples above, fro..."
829,attention_is_all_you_need,15,112,at layer 5 of 6. The heads clearly learned to ...


In [99]:
df_filtered.to_parquet("data/debug_read_pdf.parquet", engine="pyarrow")